In [1]:
#ProgramName: Daily-07-Load-Min-Max-DATE-results
#Purpose:     Run collection routine for min-max details about system (Quality Assurance)
#Author:      Greg Turmel, Director, Data Governance 
#Date:        2020.08.30 - 2021.06.30
#Errata:      0.1 Improvements can be made to script using for/looping through the databases

import os, sys, argparse, pyodbc, sql, time, datetime
from datetime import timedelta
import errno, pathlib2
import sqlalchemy as db

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

from io import StringIO
import csv
import re

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

now01 = datetime.datetime.now()
print('0.0.0 Start: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

0.0.0 Start:  2021-06-14 08:33:44


In [2]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running

now01 = datetime.datetime.now()
print('1.0.0 Begin folder-subfolder check: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'MinMax\Process'
bn = r'MinMax\Reports'
ln = r'MinMax\mmLoad'
sn = r'MinMax\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathMP = os.path.join(parentDirectory, programDirectory, an)
pathMR = os.path.join(parentDirectory, programDirectory, bn)
pathMS = os.path.join(parentDirectory, programDirectory, sn)
pathMT = os.path.join(parentDirectory, programDirectory, tn)
pathML = os.path.join(parentDirectory, programDirectory, ln)

procpath = pathMP
csvpath = pathMR
sqlpath = pathMS
configpath = pathMT
loadpath = pathML

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory,           '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMP):
        os.makedirs(pathMP, mode)
        print('Program Directory subfolder has been created: ', pathMP)
    else:
        print('Directory:', pathMP,                     '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMR):
        os.makedirs(pathMR, mode)
        print('Program Directory subfolder has been created: ', pathMR)
    else:
        print('Directory:', pathMR,                     '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMS):
        os.makedirs(pathMS, mode)
        print('Program Directory subfolder has been created: ', pathMS)
    else:
        print('Directory:', pathMS,                     '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathML):
        os.makedirs(pathML, mode)
        print('Program Directory subfolder has been created: ', pathML)
    else:
        print('Directory:', pathML,                      '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMT):
        os.makedirs(pathMT, mode)
        print('Program Directory subfolder has been created: ', pathMT)
    else:
        print('Central Config Directory:', pathMT,      '>>>>> Note ---- folder already exists <<<<<')
except OSError as error:
    print(error)
    pass


now01 = datetime.datetime.now()
print('1.0.0 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.0.0 Begin folder-subfolder check:  2021-06-14 08:33:44
Parent Directory is:  C:\Users\e201873842\Documents\Py
Directory: Daily >>>>> Note ---- folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\Process >>>>> Note ---- folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\Reports >>>>> Note ---- folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\sql >>>>> Note ---- folder already exists <<<<<
Directory: C:\Users\e201873842\Documents\Py\Daily\MinMax\mmLoad >>>>> Note ---- folder already exists <<<<<
Central Config Directory: C:\Users\e201873842\Documents\Py\Daily\config >>>>> Note ---- folder already exists <<<<<
1.0.0 Processing Step Complete:  2021-06-14 08:33:44


In [3]:
# ===================================================================================================================
# Variables: Program set up
# ===================================================================================================================
#0. Set and specify a file name for each variable 
#1. Establish if prod or test run using csvtype, debugging using Debug variable, and set time variable to track process
#2. Read into dataframe and establish database names, schema names, and table names 
#3. For debugging issues: Write dataframe results into an output csv 
#4. Validate database is available today and then create dynamic sql file(s) for processing
now01 = datetime.datetime.now()
print('1.1.0 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT DatabaseName FROM Greg.dbo.Master WHERE Core = 'Core' ORDER BY DatabaseName
                              '''
                              ,conn) # here, the 'conn' is the variable 

df_DatabaseNameC = pd.DataFrame(sql_query)

conn.close()

now01 = datetime.datetime.now()
print('1.1.0 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.0 Begin Processing:  2021-06-14 08:33:44
1.1.0 Processing Step Complete:  2021-06-14 08:33:44


In [4]:
now01 = datetime.datetime.now()
print('1.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT DatabaseName FROM Greg.dbo.Master WHERE Core = 'Non' ORDER BY DatabaseName
                              '''
                              ,conn) # 

df_DatabaseNameNC = pd.DataFrame(sql_query)
  
conn.close()

now01 = datetime.datetime.now()
print('1.1.1 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.1 Begin Processing:  2021-06-14 08:33:44
1.1.1 Processing Step Complete:  2021-06-14 08:33:44


In [5]:
now01 = datetime.datetime.now()
print('1.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT SchemaName FROM Greg.dbo.Master WHERE SchemaName NOT LIKE 'SchoolS\E200503809' AND Core = 'Core' ORDER BY SchemaName
                              '''
                              ,conn) # 

df_SchemaNameC = pd.DataFrame(sql_query)

conn.close()

now01 = datetime.datetime.now()
print('1.1.2 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.2 Begin Processing:  2021-06-14 08:33:44
1.1.2 Processing Step Complete:  2021-06-14 08:33:44


In [6]:
now01 = datetime.datetime.now()
print('1.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT SchemaName FROM Greg.dbo.Master WHERE SchemaName NOT LIKE 'SCHOOLS\E200503809' AND Core = 'Non' ORDER BY SchemaName
                              '''
                              ,conn) # 

df_SchemaNameNC = pd.DataFrame(sql_query)
 
conn.close()

now01 = datetime.datetime.now()
print('1.1.3 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.3 Begin Processing:  2021-06-14 08:33:44
1.1.3 Processing Step Complete:  2021-06-14 08:33:44


In [7]:
now01 = datetime.datetime.now()
print('1.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT TableName FROM Greg.dbo.Master WHERE Core = 'Core' ORDER BY TableName
                              '''
                              ,conn) #

df_TableNameC = pd.DataFrame(sql_query)

conn.close()

now01 = datetime.datetime.now()
print('1.1.4 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.4 Begin Processing:  2021-06-14 08:33:44
1.1.4 Processing Step Complete:  2021-06-14 08:33:44


In [8]:
now01 = datetime.datetime.now()
print('1.1.5 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

csvtype = 'Base' #csvtype - use 'Test or Base'

conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')

sql_query = pd.read_sql_query(''' 
select DISTINCT TableName FROM Greg.dbo.Master WHERE Core = 'Non' ORDER BY TableName
                              '''
                              ,conn) #

df_TableNameNC = pd.DataFrame(sql_query)

conn.close()

now01 = datetime.datetime.now()
print('1.1.5 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

1.1.5 Begin Processing:  2021-06-14 08:33:44
1.1.5 Processing Step Complete:  2021-06-14 08:33:44


In [9]:
#Locate/Relocate the driving xls/csv core-noncore file structure based on your system
#Then: Read the excel file(s) with core and noncore source tables 
#(control input using csvtype variable "base" and "test") 
#f1a = pd.read_csv("{}/{}".format(pathMT, 'coreSource'+ csvtype +'.csv'), encoding='1252')
#f1b = pd.read_csv("{}/{}".format(pathMT, 'noncoreSource'+ csvtype +'.csv'), encoding='1252')

now01 = datetime.datetime.now()
print('2.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Set Core dataframes
f1c = df_DatabaseNameC['DatabaseName']        #(Core) Create dataframe with what is found in f1 dataframe
f1d = df_TableNameC['TableName']              #(Core) Create new dataframe with tables found in f1 dataframe
f1g = df_SchemaNameC['SchemaName']            #(Core) Create new dataframe with tables found in f1 dataframe

#Set NONCore dataframes
f1e = df_DatabaseNameNC['DatabaseName']       #(Non) Create dataframe with what is found in f1 dataframe
f1f = df_TableNameNC['TableName']             #(Non) Create new dataframe with tables found in f1 dataframe
f1h = df_SchemaNameNC['SchemaName']           #(Non) Create new dataframe with tables found in f1 dataframe

#De-Dupe Core dataframe contents
dbListC = f1c.drop_duplicates()         #de-dupe
tblListC = f1d.drop_duplicates()        #de-dupe
schemaListC = f1g.drop_duplicates()     #de-dupe

#Sort Core dataframe contents
dbListC = dbListC.sort_values()            #Sort
tblListC = tblListC.sort_values()          #Sort
schemaListC = schemaListC.sort_values()    #Sort

#De-Dupe NONCore dataframe contents
dbListN = f1e.drop_duplicates()         #de-dupe
tblListN = f1f.drop_duplicates()        #de-dupe
schemaListN = f1h.drop_duplicates()     #de-dupe

#Sort NONCore dataframe contents
dbListN = dbListN.sort_values()            #Sort
tblListN = tblListN.sort_values()          #Sort
schemaListN = schemaListN.sort_values()    #Sort

#Debug Control
Dbug = 'Off'     #Debugging - use On or Off  
if Dbug == 'On':
    print (dbListC)
    print (dbListN)
    #break

# ===================================================================================================================
# Setup Dynamic query seperation: CORE
# USE 'CD' for core DATE, 'CI' for core INT, USE 'CC' for core CHAR, VARCHAR, NCHAR, NVARCHAR 
# ===================================================================================================================
file04 = 'mmDynamicLoadCD.sql'      # CORE - ALL
file04a = 'mmDynamicLoadCDa.sql'    # CORE - Application Data
file04b = 'mmDynamicLoadCDb.sql'    # CORE - Assessment
file04c = 'mmDynamicLoadCDc.sql'    # CORE - GSDR

#file05 = 'mmDynamicLoadNCD.sql'     # NON core - All
file05a = 'mmDynamicLoadNCDa.sql'   # NON core - Application Data
file05b = 'mmDynamicLoadNCDb.sql'   # NON core - Assessment
file05c = 'mmDynamicLoadNCDc.sql'   # NON core - Control
file05d = 'mmDynamicLoadNCDd.sql'   # NON core - DownstreamFeeds
file05e = 'mmDynamicLoadNCDe.sql'   # NON core - DownstreamFeeds_V1.5
file05f = 'mmDynamicLoadNCDf.sql'   # NON core - GCPS_Operations
file05g = 'mmDynamicLoadNCDg.sql'   # NON core - GSDR
file05h = 'mmDynamicLoadNCDh.sql'   # NON core - GSDR_Synergy
file05i = 'mmDynamicLoadNCDi.sql'   # NON core - GSDR_Synergy_Temp
file05j = 'mmDynamicLoadNCDj.sql'   # NON core - GSDR_Temp
file05k = 'mmDynamicLoadNCDk.sql'   # NON core - ODS_WebApps
file05l = 'mmDynamicLoadNCDl.sql'   # NON core - PPROD
file05m = 'mmDynamicLoadNCDm.sql'   # NON core - Predictive_Analytics

file05n = 'mmDynamicLoadNCDa.sql'   # NON core - Application Data
file05o = 'mmDynamicLoadNCDb.sql'   # NON core - Assessment
file05p = 'mmDynamicLoadNCDc.sql'   # NON core - Control
file05q = 'mmDynamicLoadNCDd.sql'   # NON core - DownstreamFeeds
file05r = 'mmDynamicLoadNCDe.sql'   # NON core - DownstreamFeeds_V1.5
file05s = 'mmDynamicLoadNCDf.sql'   # NON core - GCPS_Operations
file05t = 'mmDynamicLoadNCDg.sql'   # NON core - GSDR
file05u = 'mmDynamicLoadNCDh.sql'   # NON core - GSDR_Synergy
file05v = 'mmDynamicLoadNCDi.sql'   # NON core - GSDR_Synergy_Temp
file05w = 'mmDynamicLoadNCDj.sql'   # NON core - GSDR_Temp
file05x = 'mmDynamicLoadNCDk.sql'   # NON core - ODS_WebApps
file05y = 'mmDynamicLoadNCDl.sql'   # NON core - PPROD
file05z = 'mmDynamicLoadNCDm.sql'   # NON core - Predictive_Analytics

file06 = 'mmLoadReport.csv'
# ===================================================================================================================
file10 = 'mmLoadMergeC.csv'
file11 = 'mmLoadMergeN.csv'
file12 = 'mmLoadMerge.csv'
# ===================================================================================================================
# Specify a path above for reports: Make sure output location is available for write 
# ===================================================================================================================
#CORE Load
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file04a), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file04b), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file04c), 'w') as fp: 
    pass

#NON-core DATE Load
#with open(os.path.join(pathMS, file05), 'w') as fp: 
    #pass
with open(os.path.join(pathMS, file05a), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05b), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05c), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05d), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05e), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05f), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05g), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05h), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05i), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05j), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05k), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05l), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05m), 'w') as fp: 
    pass

#NON-core
with open(os.path.join(pathMS, file05n), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05o), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05p), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05q), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05r), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05s), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05t), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05u), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05v), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05w), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05x), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05y), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05z), 'w') as fp: 
    pass

#Reporting
with open(os.path.join(pathMR, file06), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file10), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file11), 'w') as fp: 
    pass
with open(os.path.join(pathMR, file12), 'w') as fp: 
    pass

now01 = datetime.datetime.now()
print('2.1.1 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

2.1.1 Begin Processing:  2021-06-14 08:33:44
2.1.1 Processing Step Complete:  2021-06-14 08:33:44


In [10]:
# ===================================================================================================================
#  MinMax (DATE) Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('3.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=ApplicationData')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file04a), 'a') as f:
    df.to_string(f, header=None, index=False)
    
sql.close()
conn.close()

now01 = datetime.datetime.now()
print('3.1.1 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

3.1.1 Begin Processing:  2021-06-14 08:33:44
3.1.1 Processing Step Complete:  2021-06-14 08:33:45


In [11]:
# ===================================================================================================================
#  MinMax (DATE) Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('3.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Assessment')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file04b), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()

now01 = datetime.datetime.now()
print('3.1.2 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

3.1.2 Begin Processing:  2021-06-14 08:33:45
3.1.2 Processing Step Complete:  2021-06-14 08:33:45


In [12]:
# ===================================================================================================================
#  MinMax (DATE) Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('3.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file04c), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()

now01 = datetime.datetime.now()
print('3.1.3 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

3.1.3 Begin Processing:  2021-06-14 08:33:45
3.1.3 Processing Step Complete:  2021-06-14 08:33:46


#Begin Review of methods for writing out files ***********************************************************************

In [13]:
now01 = datetime.datetime.now()
print('3.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

# Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadCDa.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('3.1.4 Processing Dynamic Min-Max C DATE SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=ApplicationData')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file10), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()
        
now01 = datetime.datetime.now()
print('3.1.4 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

3.1.4 Begin Processing:  2021-06-14 08:33:46
3.1.4 Processing Dynamic Min-Max C DATE SQL:  2021-06-14 08:33:46
3.1.4 Processing Step Complete:  2021-06-14 08:33:46


In [14]:
now01 = datetime.datetime.now()
print('3.1.5 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

# Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadCDb.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('3.1.5 Processing Dynamic Min-Max C DATE SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Assessment')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file10), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()
        
now01 = datetime.datetime.now()
print('3.1.5 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

3.1.5 Begin Processing:  2021-06-14 08:33:46
3.1.5 Processing Dynamic Min-Max C DATE SQL:  2021-06-14 08:33:46
3.1.5 Processing Step Complete:  2021-06-14 08:33:51


In [15]:
now01 = datetime.datetime.now()
print('3.1.6 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

# Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadCDc.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
now01 = datetime.datetime.now()
print('3.1.6 Processing Dynamic Min-Max C DATE SQL: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file10), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()
        
now01 = datetime.datetime.now()
print('3.1.6 Processing Step Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")  

3.1.6 Begin Processing:  2021-06-14 08:33:51
3.1.6 Processing Dynamic Min-Max C DATE SQL:  2021-06-14 08:33:51
3.1.6 Processing Step Complete:  2021-06-14 08:33:51


In [16]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=ApplicationData')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05a), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.1 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.1 Begin Processing:  2021-06-14 08:33:52
4.1.1 Processing Complete:  2021-06-14 08:33:52


In [17]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=Assessment')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05b), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.2 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.2 Begin Processing:  2021-06-14 08:33:52
4.1.2 Processing Complete:  2021-06-14 08:33:53


In [18]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=Control')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05c), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.3 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.3 Begin Processing:  2021-06-14 08:33:53
4.1.3 Processing Complete:  2021-06-14 08:33:54


In [19]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=DownstreamFeeds')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05d), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.4 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.4 Begin Processing:  2021-06-14 08:33:54
4.1.4 Processing Complete:  2021-06-14 08:33:56


In [20]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.5 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=DownstreamFeeds_V1.5')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05e), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.5 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.5 Begin Processing:  2021-06-14 08:33:56
4.1.5 Processing Complete:  2021-06-14 08:33:56


In [21]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.6 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=GCPS_Operations')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05f), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.6 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.6 Begin Processing:  2021-06-14 08:33:56
4.1.6 Processing Complete:  2021-06-14 08:33:57


In [22]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.7 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=GSDR')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05g), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.7 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.7 Begin Processing:  2021-06-14 08:33:57
4.1.7 Processing Complete:  2021-06-14 08:33:58


In [23]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.8 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=GSDR_Synergy')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05h), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.8 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.8 Begin Processing:  2021-06-14 08:33:58
4.1.8 Processing Complete:  2021-06-14 08:33:59


In [24]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.9 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=GSDR_Synergy_Temp')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05i), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.9 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.9 Begin Processing:  2021-06-14 08:33:59
4.1.9 Processing Complete:  2021-06-14 08:33:59


In [25]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.10 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=GSDR_Temp')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05j), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.10 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.10 Begin Processing:  2021-06-14 08:33:59
4.1.10 Processing Complete:  2021-06-14 08:34:01


In [26]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.11 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=ODS_WebApps')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05k), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.11 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.11 Begin Processing:  2021-06-14 08:34:01
4.1.11 Processing Complete:  2021-06-14 08:34:01


In [27]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.12 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=PPROD')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05l), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.12 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.12 Begin Processing:  2021-06-14 08:34:01
4.1.12 Processing Complete:  2021-06-14 08:34:01


In [28]:
# ===================================================================================================================
#  MinMax (DATE) Non-Core - Create Dynamic SQL - begin programming steps
# ===================================================================================================================
now01 = datetime.datetime.now()
print('4.1.13 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

sql = open(os.path.join(pathMS, 'mmNCDload.sql'))

conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={SQL Server};'
                              'Database=Predictive_Analytics')

df = pd.read_sql_query(sql.read(), conn)

with open(os.path.join(pathMS, file05m), 'a') as f:
    df.to_string(f, header=None, index=False)

sql.close()
conn.close()
now01 = datetime.datetime.now()
print('4.1.13 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

4.1.13 Begin Processing:  2021-06-14 08:34:01
4.1.13 Processing Complete:  2021-06-14 08:34:03


In [29]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=30)
print('5.1.1 Expected Processing time +30 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDa.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=ApplicationData')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.1 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.1 Connection Closed - Report Generated - Program Completed ****')

5.1.1 Begin Processing:  2021-06-14 08:34:03
5.1.1 Expected Processing time +30 min: @ 2021-06-14 09:04:03
5.1.1 Processing Complete:  2021-06-14 09:06:09
5.1.1 Connection Closed - Report Generated - Program Completed ****


In [30]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.2 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDb.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Assessment')
for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.2 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.2 Connection Closed - Report Generated - Program Completed ****')

5.1.2 Begin Processing:  2021-06-14 09:06:09
5.1.2 Expected Processing +1 min: @ 2021-06-14 09:07:09
5.1.2 Processing Complete:  2021-06-14 09:07:09
5.1.2 Connection Closed - Report Generated - Program Completed ****


In [31]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=2)
print('5.1.3 Expected Processing +2 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDc.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=Control')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.3 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.3 Connection Closed - Report Generated - Program Completed ****')

5.1.3 Begin Processing:  2021-06-14 09:07:09
5.1.3 Expected Processing +2 min: @ 2021-06-14 09:09:09
5.1.3 Processing Complete:  2021-06-14 09:08:47
5.1.3 Connection Closed - Report Generated - Program Completed ****


In [32]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=15)
print('5.1.4 Expected Processing +15 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDd.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=DownstreamFeeds')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.4 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.4 Connection Closed - Report Generated - Program Completed ****')

5.1.4 Begin Processing:  2021-06-14 09:08:47
5.1.4 Expected Processing +15 min: @ 2021-06-14 09:23:47
5.1.4 Processing Complete:  2021-06-14 09:24:05
5.1.4 Connection Closed - Report Generated - Program Completed ****


In [33]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.5 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.5 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDe.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=DownstreamFeeds_V1.5')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.5 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.5 Connection Closed - Report Generated - Program Completed ****')

5.1.5 Begin Processing:  2021-06-14 09:24:05
5.1.5 Expected Processing +1 min: @ 2021-06-14 09:25:05
5.1.5 Processing Complete:  2021-06-14 09:24:08
5.1.5 Connection Closed - Report Generated - Program Completed ****


In [34]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.6 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.6 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDf.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GCPS_Operations')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.6 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.6 Connection Closed - Report Generated - Program Completed ****')

5.1.6 Begin Processing:  2021-06-14 09:24:08
5.1.6 Expected Processing +1 min: @ 2021-06-14 09:25:08
5.1.6 Processing Complete:  2021-06-14 09:24:12
5.1.6 Connection Closed - Report Generated - Program Completed ****


In [35]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.7 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=2)
print('5.1.7 Expected Processing +2 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDg.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.7 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.7 Connection Closed - Report Generated - Program Completed ****')

5.1.7 Begin Processing:  2021-06-14 09:24:12
5.1.7 Expected Processing +2 min: @ 2021-06-14 09:26:12
5.1.7 Processing Complete:  2021-06-14 09:25:23
5.1.7 Connection Closed - Report Generated - Program Completed ****


In [36]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.8 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.8 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDh.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR_Synergy')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.8 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.8 Connection Closed - Report Generated - Program Completed ****')

5.1.8 Begin Processing:  2021-06-14 09:25:23
5.1.8 Expected Processing +1 min: @ 2021-06-14 09:26:23
5.1.8 Processing Complete:  2021-06-14 09:25:23
5.1.8 Connection Closed - Report Generated - Program Completed ****


In [37]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.9 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.9 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDi.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR_Synergy_Temp')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.9 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.9 Connection Closed - Report Generated - Program Completed ****')

5.1.9 Begin Processing:  2021-06-14 09:25:23
5.1.9 Expected Processing +1 min: @ 2021-06-14 09:26:23
5.1.9 Processing Complete:  2021-06-14 09:25:24
5.1.9 Connection Closed - Report Generated - Program Completed ****


In [38]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.10 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.10 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDj.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=GSDR_Temp')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.10 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.10 Connection Closed - Report Generated - Program Completed ****')

5.1.10 Begin Processing:  2021-06-14 09:25:24
5.1.10 Expected Processing +1 min: @ 2021-06-14 09:26:24
5.1.10 Processing Complete:  2021-06-14 09:26:33
5.1.10 Connection Closed - Report Generated - Program Completed ****


In [39]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.11 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.11 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDk.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                      'Trusted_Connection=yes;'
                      'DRIVER={SQL Server};'
                      'Database=ODS_WebApps')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.11 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.11 Connection Closed - Report Generated - Program Completed ****')

5.1.11 Begin Processing:  2021-06-14 09:26:33
5.1.11 Expected Processing +1 min: @ 2021-06-14 09:27:33
5.1.11 Processing Complete:  2021-06-14 09:26:58
5.1.11 Connection Closed - Report Generated - Program Completed ****


In [40]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.12 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('5.1.12 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDl.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                        'Trusted_Connection=yes;'
                        'DRIVER={SQL Server};'
                        'Database=PPROD')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.12 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.12 Connection Closed - Report Generated - Program Completed ****')

5.1.12 Begin Processing:  2021-06-14 09:26:58
5.1.12 Expected Processing +1 min: @ 2021-06-14 09:27:58
5.1.12 Processing Complete:  2021-06-14 09:27:54
5.1.12 Connection Closed - Report Generated - Program Completed ****


In [41]:
# ===================================================================================================================
#  MinMax (DATE) NonCore Processing - begin processing dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.13 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=40)
print('5.1.13 Expected Processing +40 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic NC sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCDm.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
conn = pyodbc.connect('Server=PRODODSSQL;'
                        'Trusted_Connection=yes;'
                        'DRIVER={SQL Server};'
                        'Database=Predictive_Analytics')

for sqlCmd in sqlFile.split(';'):
    try:
        #cursor = conn.cursor()
        df = pd.read_sql_query(sqlCmd, conn)
        with open(os.path.join(pathMR, file06), 'a') as f:
            df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
        with open(os.path.join(pathMR, file11), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        with open(os.path.join(pathMR, file12), 'a') as f:
            df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
    finally:
        #cursor.close()
        continue
        
# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('5.1.13 Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('5.1.13 Connection Closed - Report Generated - Program Completed ****')

5.1.13 Begin Processing:  2021-06-14 09:27:54
5.1.13 Expected Processing +40 min: @ 2021-06-14 10:07:54
5.1.13 Processing Complete:  2021-06-14 10:07:34
5.1.13 Connection Closed - Report Generated - Program Completed ****


In [42]:
#Core Type 
#Variables: 
now = datetime.datetime.now()
csvtype = 'Base' #csvtype - use 'Test or Base'
Dbug = 'Off'     #Debugging - use On or Off  

# ===================================================================================================================
# Dynamic query seperation: CORE
# USE 'CI' for CORE INT, USE 'CC' for CORE CHAR, USE 'CV' for VARCHAR, USE 'CNV' for NVARCHAR, USE 'CN' for NCHAR 
# ===================================================================================================================
file04 = 'mmDynamicLoadCI.sql'
file05 = 'mmDynamicLoadNCI.sql'
file06 = 'mmLoadReport.csv'
# ===================================================================================================================
file12 = 'mmLoadMerge.csv'
# ===================================================================================================================
now01 = datetime.datetime.now()
print('6.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Creating a report file to load todays data into at the specified location 
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05), 'w') as fp: 
    pass

#Min (INT) Core - Create Dynamic SQL
for x in dbListC:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmCIload.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file04), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

now01 = datetime.datetime.now()
print('6.1.1 Create Dynamic Core SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

6.1.1 Begin Processing:  2021-06-14 10:07:34
6.1.1 Create Dynamic Core SQL Complete:  2021-06-14 10:07:35


In [43]:
# ===================================================================================================================
#  MinMax (INT) Core Processing 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('6.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadCI.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
for sqlCmd in sqlFile.split(';'):
    for x in dbListC:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))

            df = pd.read_sql_query(sqlCmd, conn)
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        except:
            continue
        finally:
            continue

# Close connection to database
conn.close()

now01 = datetime.datetime.now()
print('6.1.2 Processing Dynamic Core Min-Max INT Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

6.1.2 Begin Processing:  2021-06-14 10:07:35
6.1.2 Processing Dynamic Core Min-Max INT Complete:  2021-06-14 10:08:03


In [44]:
# ===================================================================================================================
#  MinMax (INT) NonCore - Create Dynamic SQL 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('6.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for x in dbListN:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmNCIload.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file05), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

now01 = datetime.datetime.now()
print('6.1.3 Create Dynamic Non Core INT SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

6.1.3 Begin Processing:  2021-06-14 10:08:03
6.1.3 Create Dynamic Non Core INT SQL Complete:  2021-06-14 10:08:16


In [45]:
# ===================================================================================================================
#  MinMax (INT) Non-Core Processing 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('6.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=10)
print('6.1.4 Expected Processing +10 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCI.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
for sqlCmd in sqlFile.split(';'):
    for x in dbListN:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))

            df = pd.read_sql_query(sqlCmd, conn)
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        except:
            continue
        finally:
            continue

# Close connection to database
conn.close()

# ===================================================================================================================
now01 = datetime.datetime.now()
print('6.1.4 NonCore INT Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('6.1.4 Connection Closed - Report Generated - Program Completed ****')

6.1.4 Begin Processing:  2021-06-14 10:08:16
6.1.4 Expected Processing +10 min: @ 2021-06-14 10:18:16
6.1.4 NonCore INT Processing Complete:  2021-06-14 10:18:24
6.1.4 Connection Closed - Report Generated - Program Completed ****


In [46]:
# Core Type 
# Variables: 
# ===================================================================================================================
#  Dynamic query seperation: CORE
#  USE 'CI' for INT, USE 'CC' for CHAR, USE 'CV' for VARCHAR, USE 'CNV' for NVARCHAR, USE 'CN' for NCHAR 
# ===================================================================================================================
file04 = 'mmDynamicLoadCC.sql'
file05 = 'mmDynamicLoadNCC.sql'
file06 = 'mmLoadReport.csv'
# ===================================================================================================================
file12 = 'mmLoadMerge.csv'
# ===================================================================================================================

#Creating a report file to load todays data into at the specified location 
with open(os.path.join(pathMS, file04), 'w') as fp: 
    pass
with open(os.path.join(pathMS, file05), 'w') as fp: 
    pass
# ===================================================================================================================
#  MinMax (CHAR) Core - Create Dynamic SQL
# ===================================================================================================================
now01 = datetime.datetime.now()
print('7.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for x in dbListC:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmCCload.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file04), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

now01 = datetime.datetime.now()
print('7.1.1 Create Dynamic SQL Core Processing Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

7.1.1 Begin Processing:  2021-06-14 10:18:24
7.1.1 Create Dynamic SQL Core Processing Complete:  2021-06-14 10:18:25


In [47]:
# ===================================================================================================================
#  MinMax (CHAR) Core Processing 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('7.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadCC.sql'))
sqlFile = s.read()
s.close()


#strip and then split read on';' 
for sqlCmd in sqlFile.split(';'):
    for x in dbListC:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            #cursor = conn.cursor()
            df = pd.read_sql_query(sqlCmd, conn)
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        except:
            continue
        finally:
            continue

# Close connection to database
conn.close()

now01 = datetime.datetime.now()
print('7.1.2 Processing Dynamic Core Min-Max C CHAR SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

7.1.2 Begin Processing:  2021-06-14 10:18:26
7.1.2 Processing Dynamic Core Min-Max C CHAR SQL Complete:  2021-06-14 10:18:33


In [48]:
# ===================================================================================================================
#  MinMax (CHAR) NonCore - Create Dynamic SQL 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('7.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

for x in dbListN:
    try:
        conn = pyodbc.connect('Server=PRODODSSQL;'
                              'Trusted_Connection=yes;'
                              'DRIVER={{SQL Server}};'
                              'Database={0}'.format(x))

        sql = open(os.path.join(pathMS, 'mmNCCload.sql'))
        df = pd.read_sql_query(sql.read(), conn)

        with open(os.path.join(pathMS, file05), 'a') as f:
            df.to_string(f, header=None, index=False)

        sql.close()
        conn.close()
    finally:
        continue

now01 = datetime.datetime.now()
print('7.1.3 NonCore Processing Dynamic Min-Max C CHAR SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
#Print (" ")

7.1.3 Begin Processing:  2021-06-14 10:18:33
7.1.3 NonCore Processing Dynamic Min-Max C CHAR SQL Complete:  2021-06-14 10:18:45


In [49]:
# ===================================================================================================================
#  MinMax (CHAR) NonCore Processing 
# ===================================================================================================================
now01 = datetime.datetime.now()
print('7.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=5)
print('7.1.4 Expected Processing +5 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Read into the df the dynamic sql file built above
s = open(os.path.join(pathMS, 'mmDynamicLoadNCC.sql'))
sqlFile = s.read()
s.close()

#strip and then split read on';' 
for sqlCmd in sqlFile.split(';'):
    for x in dbListN:
        try:
            conn = pyodbc.connect('Server=PRODODSSQL;'
                                  'Trusted_Connection=yes;'
                                  'DRIVER={{SQL Server}};'
                                  'Database={0}'.format(x))
            #cursor = conn.cursor()
            df = pd.read_sql_query(sqlCmd, conn)
            with open(os.path.join(pathMR, file06), 'a') as f:
                df.to_csv(f, header=f.tell()==0, index=False, line_terminator='\n')
            with open(os.path.join(pathMR, file11), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
            with open(os.path.join(pathMR, file12), 'a') as f:
                df.to_csv(f, header=f.tell()==0, sep=',', index=False, encoding='cp1252', line_terminator='\n')
        except:
            continue
        finally:
            continue

# Close connection to database
conn.close()
# ===================================================================================================================
now01 = datetime.datetime.now()
print('7.1.4 Processing Dynamic Min-Max NC CHAR SQL Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
# ===================================================================================================================
print('7.1.4 Connection Closed - Report Generated - Program Completed ****')

7.1.4 Begin Processing:  2021-06-14 10:18:45
7.1.4 Expected Processing +5 min: @ 2021-06-14 10:23:45
7.1.4 Processing Dynamic Min-Max NC CHAR SQL Complete:  2021-06-14 10:23:43
7.1.4 Connection Closed - Report Generated - Program Completed ****


In [50]:
now01 = datetime.datetime.now()
print('8.0.0 End: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

print('8.0.0 Min-Max Date data generated - This Section of the Py Notebook Program has Completed ****')

8.0.0 End:  2021-06-14 10:23:43
8.0.0 Min-Max Date data generated - This Section of the Py Notebook Program has Completed ****


In [51]:
now01 = datetime.datetime.now()
print('9.1.0 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.0 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Set up table - remove previous records 
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute(''' 
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF EXISTS(SELECT 1 FROM dbo.MinMaxDateCore)
BEGIN
   DELETE FROM Greg.dbo.MinMaxDateCore WHERE [TodaysDate] > '19000101';
END
'''
,)
conn.commit()

conn.close()

print('9.1.3 Note:  database table Greg.dbo.MinMaxDateCore cleaned out - step complete ***')

# Import CSV
data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMergeC.csv'))   
# TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDateCore (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.0 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDateCore
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.0 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.0 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

9.1.0 Begin Processing:  2021-06-14 10:23:43
9.1.0 Expected Processing +1 min: @ 2021-06-14 10:24:43
9.1.3 Note:  database table Greg.dbo.MinMaxDateCore cleaned out - step complete ***
9.1.0 Note: csv file written to database table complete ***
9.1.0 Note: count from table data loaded is: ***      
0  17
9.1.0 Processing CSV data import Complete:  2021-06-14 10:23:43


In [52]:
now01 = datetime.datetime.now()
print('9.1.0 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.0 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

# Import CSV
data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMergeC.csv'))   
# TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDateCoreTracking (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.0 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDateCoreTracking
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.0 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.0 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

9.1.0 Begin Processing:  2021-06-14 10:23:43
9.1.0 Expected Processing +1 min: @ 2021-06-14 10:24:43
9.1.0 Note: csv file written to database table complete ***
9.1.0 Note: count from table data loaded is: ***      
0  17
9.1.0 Processing CSV data import Complete:  2021-06-14 10:23:43


In [53]:
now01 = datetime.datetime.now()
print('9.1.1 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.1 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Set up table - remove previous records 
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute(''' 
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF EXISTS(SELECT 1 FROM dbo.MinMaxDateNonCore)
BEGIN
   DELETE FROM Greg.dbo.MinMaxDateNonCore WHERE [TodaysDate] > '19000101';
END
'''
,)
conn.commit()

conn.close()

print('9.1.3 Note:  database table Greg.dbo.MinMaxDateNonCore cleaned out - step complete ***')

# Import CSV
print('9.1.3 Begin loading todays records into the reporting table ***')
data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMergeN.csv'))   
#TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])
df2 = df.fillna(value=0)
df = df2

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDateNonCore (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.1 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDateNonCore
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.1 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.1 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))


9.1.1 Begin Processing:  2021-06-14 10:23:43
9.1.1 Expected Processing +1 min: @ 2021-06-14 10:24:43
9.1.3 Note:  database table Greg.dbo.MinMaxDateNonCore cleaned out - step complete ***
9.1.3 Begin loading todays records into the reporting table ***
9.1.1 Note: csv file written to database table complete ***
9.1.1 Note: count from table data loaded is: ***        
0  4656
9.1.1 Processing CSV data import Complete:  2021-06-14 10:24:21


In [54]:
now01 = datetime.datetime.now()
print('9.1.2 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.2 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

# Import CSV
data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMergeN.csv'))   
#TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])
df2 = df.fillna(value=0)
df = df2

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDateNonCoreTracking (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.2 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDateNonCoreTracking
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.2 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.2 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))


9.1.2 Begin Processing:  2021-06-14 10:24:21
9.1.2 Expected Processing +1 min: @ 2021-06-14 10:25:21
9.1.2 Note: csv file written to database table complete ***
9.1.2 Note: count from table data loaded is: ***        
0  4656
9.1.2 Processing CSV data import Complete:  2021-06-14 10:24:34


In [55]:
now01 = datetime.datetime.now()
print('9.1.3 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))

now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.3 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

#Set up table - remove previous records 
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
cursor.execute(''' 
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF EXISTS(SELECT 1 FROM dbo.MinMaxDate)
BEGIN
   DELETE FROM Greg.dbo.MinMaxDate WHERE [TodaysDate] > '19000101';
END
'''
,)
conn.commit()

conn.close()

print('9.1.3 Note:  database table Greg.dbo.MinMaxDate cleaned out - step complete ***')

# Import CSV
print('9.1.3 Begin loading todays records into the reporting table ***')

data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMerge.csv'))   
#TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])
df2 = df.fillna(value=0)
df = df2

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDate (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.3 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDate
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.3 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.3 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))


9.1.3 Begin Processing:  2021-06-14 10:24:34
9.1.3 Expected Processing +1 min: @ 2021-06-14 10:25:34
9.1.3 Note:  database table Greg.dbo.MinMaxDate cleaned out - step complete ***
9.1.3 Begin loading todays records into the reporting table ***
9.1.3 Note: csv file written to database table complete ***
9.1.3 Note: count from table data loaded is: ***        
0  4673
9.1.3 Processing CSV data import Complete:  2021-06-14 10:25:17


In [56]:
now01 = datetime.datetime.now()
print('9.1.4 Begin Processing: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
now02 = datetime.datetime.now() + timedelta(minutes=1)
print('9.1.4 Expected Processing +1 min: @',now02.strftime("%Y-%m-%d %H:%M:%S"))

# Import CSV
data = pd.read_csv("{}/{}".format(pathMR, 'mmLoadMerge.csv'))   
#TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName
df = pd.DataFrame(data, columns= ['TodaysDate','MinDate','MaxDate','DatabaseName','SchemaName','TableName','ColumnName'])
df2 = df.fillna(value=0)
df = df2

# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

# Insert DataFrame to Table
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Greg.dbo.MinMaxDateTracking (TodaysDate,MinDate,MaxDate,DatabaseName,SchemaName,TableName,ColumnName)
                VALUES (?,?,?,?,?,?,?)
                ''',
row.TodaysDate,
row.MinDate,
row.MaxDate,
row.DatabaseName,
row.SchemaName,
row.TableName,
row.ColumnName
                )
conn.commit()

cursor.close()

print('9.1.4 Note: csv file written to database table complete ***')

sql_query = pd.read_sql_query(''' 
select count(*) from Greg.dbo.MinMaxDateTracking
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection information from above
conn.close()
print('9.1.4 Note: count from table data loaded is: ***', sql_query)

now01 = datetime.datetime.now()
print('9.1.4 Processing CSV data import Complete: ',now01.strftime("%Y-%m-%d %H:%M:%S"))


9.1.4 Begin Processing:  2021-06-14 10:25:17
9.1.4 Expected Processing +1 min: @ 2021-06-14 10:26:17
9.1.4 Note: csv file written to database table complete ***
9.1.4 Note: count from table data loaded is: ***        
0  4673
9.1.4 Processing CSV data import Complete:  2021-06-14 10:25:30


In [57]:
now01 = datetime.datetime.now()
print('10.0.0 End: ',now01.strftime("%Y-%m-%d %H:%M:%S"))
print('10.0.0 Min-Max Date data load for reporting - Completed ****')

10.0.0 End:  2021-06-14 10:25:30
10.0.0 Min-Max Date data load for reporting - Completed ****
